In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
import os
import json
import torch
import yaml
import numpy as np
import cv2
import copy
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from nerfstudio.utils.eval_utils import eval_setup
from inerf.inerf_utils import load_eval_image_into_pipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/jackal_training_data_0/plane-nerf/2024-01-14_144644"
EVAL_PATH = "/stored_data/jackal_evaluation_data"

In [5]:
config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, checkpoint_path, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_training_data_0/plane-nerf/2024-01-14_144644/nerfstudio_models/step-000009999.ckpt

In [6]:
pipeline = load_eval_image_into_pipeline(pipeline,EVAL_PATH)

/workspace/inerf/inerf/inerf_utils.py:97: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  camera_to_worlds = torch.cat([camera_to_worlds, tensor([tf]).float()], 0)


Setting up training dataset...

Caching all 300 images.

Output()

In [13]:
store_metrics = []
for camera, batch in pipeline.datamanager.fixed_indices_train_dataloader:
    outputs = pipeline.model.get_outputs_for_camera(camera=camera)
    metrics_dict, images_dict = pipeline.model.get_image_metrics_and_images(outputs,batch)    
    store_metrics.append([metrics_dict["psnr"],metrics_dict["ssim"],metrics_dict["lpips"]])

print(np.mean(store_metrics,axis=0))


/home/user/.local/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


[17.8459096   0.62048922  0.56289135]


In [16]:
#Save store_metrics

TARGET_PATH = os.path.join(MODEL_PATH, "metrics.csv")
print(TARGET_PATH)
np.savetxt(TARGET_PATH, store_metrics, delimiter=",")

/workspace/outputs/jackal_training_data_0/plane-nerf/2024-01-14_144644/metrics.csv
